In [2]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [3]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [4]:
from typing import List

from models.llm.chain import BaseChain

In [5]:
template = '''
I want you to act like a Python code generator that plots graphs. You need to write Python code based on the "plotly" library. 
I will give you the graph to draw, which can be a table or a simple description. 

Follow the three steps below.
step1. Materialize the graph to plot.
step2. Write pseudocode to draw the materialized graph.
step3. Generate the plotly library Python code based on the pseudocode.
step4. Return the path to the saved image. If no image is saved, return "No graph".

In the output, the Python code should be separated by ```.
At the end of the output, return the path to the saved image.

graph1 to draw:
===
Please plot the table below as a bar chart.
| Properties | Information |
|------------|-------------|
| **Natural Language Processing Market Size in 2022** | $15.7 billion |
| **Natural Language Processing Market Forecast 2032** | $144.9 Billion |
| **Natural Language Processing Market CAGR from 2023 to 2032** | 25.1% |
===

example1 of step1:
===
- bar chart
- Title: Natural language processing market size 2022-2032.
- With 1-year bars
- 2022: $15.7 billion
- 2032: $144.9 billion
- Annualized growth rate of 25.1
===

example1 of step2:
===
BEGIN

    IMPORT necessary libraries

    DEFINE years from 2022 to 2032
    DEFINE initial market size and growth rate

    CALCULATE market sizes for each year using growth rate

    CREATE a bar chart using years and calculated market sizes
    SET chart titles and axis titles

    SAVE the chart as an image

END
===

example1 of step3:
```python
import plotly.graph_objects as go

# Set up data
years = [f"{{year}}" for year in range(2022, 2033)]
market_size = 157  # Starting size for 2022: 157 billion dollars
growth_rate = 0.251  # Growth rate: 25.1%
market_sizes = [market_size]

# Calculate market size from 2023 to 2032
for _ in range(2022, 2032):
    market_size = market_size * (1 + growth_rate)
    market_sizes.append(market_size)

# Create a bar chart
fig = go.Figure(data=[
    go.Bar(name='NLP Market Size', x=years, y=market_sizes)
])

# Set chart layout
fig.update_layout(
    title='NLP Market Size 2022~2032',
    xaxis_title='Year',
    yaxis_title='Market Size (billion dollars)',
    yaxis_tickformat=",.0f"
)

# Save the chart as an image
fig.write_image("nlp_market_size_2022_2032.png")
```

example1 of step4: nlp_market_size_2022_2032.png

graph2 to draw:
===
{graph_to_draw}
===

example2 of step1:
'''

In [6]:
class GraphChain(BaseChain):
    def __init__(self, 
                graph_template=None, 
                input_variables:List[str]=None,
                graph_template_path='../openai_skt/models/templates/graph_prompt.json', 
                model='gpt-3.5-turbo', 
                verbose=False) -> None:
        super().__init__(template=graph_template, input_variables=input_variables, template_path=graph_template_path, model=model, verbose=verbose)

    def run(self, query:str=None):
        return super().run(graph_to_draw=query)
    
    async def arun(self,query:str=None):
        return await super().arun(graph_to_draw=query)

In [7]:
graph_chain = GraphChain(graph_template=template, input_variables=['graph_to_draw'], verbose=True)

### Tool 설정

In [8]:
from typing import Optional, Type, Any
from pydantic import BaseModel, Field

from langchain.tools import BaseTool
from langchain.tools.python.tool import PythonREPLTool

In [9]:
class GraphTool(BaseTool):
    name = "graph_tool"
    description = "A tool to draw a graph. It return image path of the graph."
    args_schema: Optional[Type[BaseModel]] = None
    """Pydantic model class to validate and parse the tool's input arguments."""

    graph_chain: Any  # 클래스 변수로 선언
    python_tool: Any  # 클래스 변수로 선언

    def __init__(self, graph_chain) -> None:
        super().__init__()
        self.graph_chain = graph_chain
        # self.python_tool = PythonREPLTool()
        self.python_tool = exec

    def _run(self, query) -> dict:
        result = self.graph_chain.run(query=query)
        code, save_path = self.parse_result(result)
        if save_path is not None:
            # result = self.python_tool._run(code)
            result = self.python_tool(code)
            return save_path
        else:
            return None
    
    async def _arun(self, query) -> dict:
        result = await self.graph_chain.arun(query=query)
        code, save_path = self.parse_result(result)
        if save_path is not None:
            # result = await self.python_tool._arun(code)
            result = await self.python_tool(code)
            return save_path
        else:
            return None
    
    def parse_result(self, result) -> str:
        start_code = result.find("```python") + len("```python")
        end_code = result.rfind("```")
        code = result[start_code:end_code].strip() # text of python code
        start_path = result.find('step4: ') + len('step4: ')
        save_path = result[start_path:].strip() if result[start_path:].strip() != 'No graph' else None
        return code, save_path

In [10]:
graph_tool = GraphTool(graph_chain=graph_chain)

In [13]:
# Exaple of Bar chart
graph_to_draw = '''
아래의 표를 차트로 그려줘. 연도별 미국 금리에 대한 표야.
| 연도   | 평균    |
|--------|--------|
| 1993년 | 3.00%  |
| 1994년 | 4.29%  |
| 1995년 | 5.81%  |
| 1996년 | 5.25%  |
| 1997년 | 5.46%  |
| 1998년 | 5.31%  |
| 1999년 | 5.04%  |
| 2000년 | 6.27%  |
| 2001년 | 3.73%  |
| 2002년 | 1.67%  |
| 2003년 | 1.10%  |
| 2004년 | 1.40%  |
| 2005년 | 3.25%  |
| 2006년 | 5.02%  |
| 2007년 | 5.00%  |
| 2008년 | 1.88%  |
| 2009년 | 0.25%  |
| 2010년 | 0.25%  |
| 2011년 | 0.25%  |
| 2012년 | 0.25%  |
| 2013년 | 0.25%  |
| 2014년 | 0.25%  |
| 2015년 | 0.27%  |
| 2016년 | 0.52%  |
| 2017년 | 1.13%  |
| 2018년 | 1.96%  |
| 2019년 | 2.25%  |
| 2020년 | 0.50%  |
| 2021년 | 0.25%  |
| 2022년 | 2.02%  |
| 2023년 | 4.63%  |

'''

In [14]:
result = graph_tool._run(query=graph_to_draw)



> Entering new LLMChain chain...
Prompt after formatting:

I want you to act like a Python code generator that plots graphs. You need to write Python code based on the "plotly" library. 
I will give you the graph to draw, which can be a table or a simple description. 

Follow the three steps below.
step1. Materialize the graph to plot.
step2. Write pseudocode to draw the materialized graph.
step3. Generate the plotly library Python code based on the pseudocode.
step4. Return the path to the saved image. If no image is saved, return "No graph".

In the output, the Python code should be separated by ```.
At the end of the output, return the path to the saved image.

graph1 to draw:
===
Please plot the table below as a bar chart.
| Properties | Information |
|------------|-------------|
| **Natural Language Processing Market Size in 2022** | $15.7 billion |
| **Natural Language Processing Market Forecast 2032** | $144.9 Billion |
| **Natural Language Processing Market CAGR from 2023 to 

In [15]:
print(result)

us_interest_rates.png
